In [1]:
import json
import requests
from bs4 import BeautifulSoup
from collections import OrderedDict
from flask import Flask
import json
from flask import render_template, request
from werkzeug import secure_filename
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
import tensorflow as tf
from keras import regularizers
from keras import layers, models
from keras.applications import VGG16
from keras import Input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras.models import load_model
import time
from flask import make_response
import ast
from urllib.parse import quote

Using TensorFlow backend.


In [2]:
graph = tf.get_default_graph()

In [18]:
#연관 검색어 추천 기능

In [3]:
#초기화
app = Flask(__name__)
@app.route("/")
def test():
    return "<h1>test server</h1>"

In [4]:
product_li = ["딸기","키위","멜론","포도","수박","토마토",
        "배","감","사과","귤","고구마","감자"]
product_url_li = ["http://search.11st.co.kr/Search.tmall?kwd=%25EB%2594%25B8%25EA%25B8%25B0",
            "http://search.11st.co.kr/Search.tmall?kwd=%25ED%2582%25A4%25EC%259C%2584",
            "http://search.11st.co.kr/Search.tmall?kwd=%25EB%25A9%259C%25EB%25A1%25A0",
            "http://search.11st.co.kr/Search.tmall?kwd=%25ED%258F%25AC%25EB%258F%2584",
            "http://search.11st.co.kr/Search.tmall?kwd=%25EC%2588%2598%25EB%25B0%2595",
            "http://search.11st.co.kr/Search.tmall?kwd=%25ED%2586%25A0%25EB%25A7%2588%25ED%2586%25A0",
            "http://search.11st.co.kr/Search.tmall?kwd=%25EB%25B0%25B0",
            "http://search.11st.co.kr/Search.tmall?kwd=%25EA%25B0%2590",
            "http://search.11st.co.kr/Search.tmall?kwd=%25EC%2582%25AC%25EA%25B3%25BC",
            "http://search.11st.co.kr/Search.tmall?kwd=%25EA%25B7%25A4",
            "http://search.11st.co.kr/Search.tmall?kwd=%25EA%25B3%25A0%25EA%25B5%25AC%25EB%25A7%2588",
            "http://search.11st.co.kr/Search.tmall?kwd=%25EA%25B0%2590%25EC%259E%2590"]
product_dic = dict(zip(product_li,product_url_li))

In [5]:
@app.route("/coupang/<string:keyword>")
def product_recommendation_model_coupang(keyword):    

    flag = 1
    
    print(keyword)
    for i in product_dic:
        if i == keyword:
            product_url = product_dic[i]
            flag = 0
                
    if flag == 0:
        pass
    else:
        new_product = [""]
        new_url = [""]
        
        file_data = OrderedDict()
        file_data["related_keywords"] = new_product
        file_data["product_url"] = new_url
        j_file = json.dumps(file_data, ensure_ascii=False,indent="\t")
        return make_response(str(j_file))
        
        
    #연관 키워드 크롤링        
    response = requests.get(product_url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    ul = soup.find("ul", {"class":"finder_lst"})

    product = []
    url = []

    for value in ul.find_all("li"):
        product.append(value.a.text.strip())
        url.append(value.a.attrs['href'])

    #연관 규칙 분석
    
    key = request.files['keyword']
    frequent_itemsets = apriori(key, min_support=0.7, use_colnames=True)
    frequent_itemsets 
    related_keywords = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
    
    new_product = []
    new_url = []
    for i in range(int(len(product)/2)):
        new_product.append(product[i])
        new_url.append(url[i])        

    result_dic = dict(zip(new_product,new_url))
    file_data = OrderedDict()
    file_data["related_keywords"] = new_product
    file_data["product_url"] = new_url
    j_file = json.dumps(file_data, ensure_ascii=False,indent="\t")

    #연관 키워드 추천
    return make_response(str(j_file))

#11번가 연관 검색어 모델
@app.route("/11st/<string:keyword>")
def product_recommendation_model_11st(keyword):    
    
    flag = 1
    
    for i in product_dic:
        if i == keyword:
            product_url = product_dic[i]
            flag = 0
            
    if flag == 0:
        pass
    else:
        new_product = [""]
        new_url = [""]
        
        file_data = OrderedDict()
        file_data["related_keywords"] = new_product
        file_data["product_url"] = new_url
        j_file = json.dumps(file_data, ensure_ascii=False,indent="\t")
        return make_response(str(j_file))
                        
    #크롤링        
    response = requests.get(product_url)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    ul = soup.find("ul", {"class":"finder_lst"})

    product = []
    url = []
        
    #json으로 반환
    for value in ul.find_all("li"):
        product.append(value.a.text.strip())
        url.append(value.a.attrs['href'])

    p_len = len(product)
    new_product = []
    new_url = []
    
    for i in range(int(len(product)/2)):
        new_product.append(product[p_len-1-i])
        new_url.append(url[p_len-1-i]) 
        
    result_dic = dict(zip(product,url))
    file_data = OrderedDict()
    file_data["related_keywords"] = new_product
    file_data["product_url"] = new_url
    j_file = json.dumps(file_data, ensure_ascii=False,indent="\t")
    
    return make_response(str(j_file))

In [ ]:
#판매 수익 예측 모델

In [107]:
#초기화
app = Flask(__name__)
@app.route("/")
def test():
    return "<h1>test server</h1>"

In [7]:
model = Sequential()
model.add(LSTM(10, return_sequences = True, input_shape=(10,1)))
model.add(LSTM(16,return_sequences = False))
model.add(Dense(1,activation='linear'))
model.compile(loss='binary_crossentropy',
      optimizer=optimizers.RMSprop(lr=1e-4),
      metrics=['acc'])

In [101]:
from tensorflow import Graph, Session
graph1 = Graph()
with graph1.as_default():
    session1 = Session()
    with session1.as_default():
        model = model

In [8]:
price = [22500, 100000, 3000, 56000, 2000, 9000, 7900, 15600, 36000, 45000, 40500, 105000, 111100, 122000, 39000, 85000, 100000, 67400, 100000, 99000, 147800, 35000, 56000, 42400, 110000, 65000, 71000, 73000, 43000, 132000, 162000, 72400, 129700, 62000, 131400, 72800, 81000, 57400, 46500, 0, 0]

price = np.asarray(price)
price_max = max(price)
price_min = min(price)
price = ((price-price_min)/(price_max-price_min))
seq_len = 10 #최근 10일에 데이터를 가지고 미래 예측
sequence_length = seq_len + 1

result = []
for index in range(len(price)-sequence_length):
    result.append(price[index:index+sequence_length])

normalized_data = []
for window in result:
    normalized_data.append(window)
result = np.array(normalized_data)

row = int(round(result.shape[0]*0.7))
train = result[:row,:]
np.random.shuffle(train)

x_train = train[:,:-1]
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
y_train = train[:,-1]

x_test = result[row:,:-1]
x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
y_test = result[row:,-1]

x_train.shape, x_test.shape
model.fit(x_train,y_train,
          validation_data=(x_test,y_test),
          batch_size=1,
          epochs=10)
preds = model.predict(x_test)

result = preds[-1]*(price_max-price_min)+price_max
price_prediction = "%0.2f"%result
print(price_prediction)


Train on 21 samples, validate on 9 samples
Epoch 1/10
21/21 [==============================] - 2s 99ms/step - loss: 5.7945 - acc: 0.0000e+00 - val_loss: 2.2941 - val_acc: 0.1111
Epoch 2/10
21/21 [==============================] - 0s 12ms/step - loss: 2.3581 - acc: 0.0000e+00 - val_loss: 1.8386 - val_acc: 0.1111
Epoch 3/10
21/21 [==============================] - 0s 12ms/step - loss: 2.0349 - acc: 0.0000e+00 - val_loss: 1.5850 - val_acc: 0.1111
Epoch 4/10
21/21 [==============================] - 0s 13ms/step - loss: 1.7964 - acc: 0.0000e+00 - val_loss: 1.3984 - val_acc: 0.1111
Epoch 5/10
21/21 [==============================] - 0s 13ms/step - loss: 1.6102 - acc: 0.0000e+00 - val_loss: 1.2590 - val_acc: 0.1111
Epoch 6/10
21/21 [==============================] - 0s 13ms/step - loss: 1.4641 - acc: 0.0000e+00 - val_loss: 1.1466 - val_acc: 0.1111
Epoch 7/10
21/21 [==============================] - 0s 14ms/step - loss: 1.3414 - acc: 0.0000e+00 - val_loss: 1.0564 - val_acc: 0.1111
Epoch 8/10


In [103]:
model.save_weights('price_predict_model_weight.h5')

In [9]:
@app.route('/price_prediction', methods=['POST'])
def price_prediction():
    value = request.get_json()
    price = value['data']
    price = np.asarray(price)
    price_max = max(price)
    price_min = min(price)
    price = ((price-price_min)/(price_max-price_min))
    print(price)

    #creaete Windows
    seq_len = 10 #최근 10일에 데이터를 가지고 미래 예측
    sequence_length = seq_len + 1

    result = []
    for index in range(len(price)-sequence_length):
        result.append(price[index:index+sequence_length])

    normalized_data = []
    for window in result:
        normalized_data.append(window)
    result = np.array(normalized_data)

    row = int(round(result.shape[0]*0.7))
    train = result[:row,:]
    np.random.shuffle(train)

    x_train = train[:,:-1]
    x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
    y_train = train[:,-1]

    x_test = result[row:,:-1]
    x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
    y_test = result[row:,-1]

    x_train.shape, x_test.shape
    #판매자, 쇼핑몰 수익예측 모델 구축
    model = Sequential()
    model.add(LSTM(10, return_sequences = True, input_shape=(10,1)))
    model.add(LSTM(16,return_sequences = False))
    model.add(Dense(1,activation='linear'))
    model.compile(loss='binary_crossentropy',
          optimizer=optimizers.RMSprop(lr=1e-4),
          metrics=['acc'])
    
    model.fit(x_train,y_train,
              validation_data=(x_test,y_test),
              batch_size=1,
              epochs=10)
    
    global graph
    with graph.as_default():
        preds = model.predict(x_test)
            
    #판매자, 쇼핑몰 수익 예측
    result = preds[-1]*(price_max-price_min)+price_max
    price_prediction = "%0.2f"%result
    print(price_prediction)
    print('prediction finish!')
    return make_response(str(price_prediction))

In [ ]:
#제품 이미지 인식 모델

In [32]:
#초기화
app = Flask(__name__)
@app.route("/")
def test():
    return "<h1>test server</h1>"

In [10]:
categories = ["사과","포도","키위","귤","멜론","배",
          "감","감자","딸기","고구마",
              "토마토","수박","unrecog"]

In [11]:
#제품 이미지 분류 모델 구축
img_model = models.Sequential()
img_model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(320, 240, 3)))
img_model.add(layers.MaxPooling2D((2, 2)))
img_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
img_model.add(layers.MaxPooling2D((2, 2)))
img_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
img_model.add(layers.MaxPooling2D((2, 2)))
img_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
img_model.add(layers.MaxPooling2D((2, 2)))
img_model.add(layers.Flatten())
img_model.add(layers.Dense(512, activation='relu'))
img_model.add(layers.Dense(13, activation='sigmoid'))
img_model.compile(loss='binary_crossentropy',
      optimizer=optimizers.RMSprop(lr=1e-4),
      metrics=['acc'])
img_model.load_weights('product_prediction_model_weight.h5')

In [8]:
img_path = "C:/Users/190713/pp_project/pp_project/"+"apple.png"
img = Image.open(img_path)
img = img.convert("RGB")
img = img.resize((240, 320))
data = np.asarray(img)
data = np.expand_dims(data, axis=0)
preds = img_model.predict(data)
print(preds)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [12]:
@app.route('/image_recognition', methods = ['GET','POST'])
def image_reocgition_model():
    if request.method == 'POST':
        f = request.files['img']
        f.save('C:/Users/190713/pp_project/pp_project/'+
               secure_filename(f.filename))
        print("img save finish!")
        
        time.sleep(3)
        
        img_path = "C:/Users/190713/pp_project/pp_project/"+f.filename
        img = Image.open(img_path)
        img = img.convert("RGB")
        img = img.resize((240, 320))
        data = np.asarray(img)
        data = np.expand_dims(data, axis=0)
        global graph
        with graph.as_default():
            preds = img_model.predict(data)
            
        print("prediction finish!")
        #제품 이미지 분류
        for i in range(0,13):
            if preds[0,i] >= 0.8:
                answer = categories[i]
        return make_response(str(answer))

In [13]:
# img model update
@app.route('/img_model_update', methods=['POST'])
def img_model_update():
    img_model = models.Sequential()
    img_model.add(layers.Conv2D(32, (3, 3), activation='relu',
                            input_shape=(320, 240, 3)))
    img_model.add(layers.MaxPooling2D((2, 2)))
    img_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    img_model.add(layers.MaxPooling2D((2, 2)))
    img_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    img_model.add(layers.MaxPooling2D((2, 2)))
    img_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    img_model.add(layers.MaxPooling2D((2, 2)))
    img_model.add(layers.Flatten())
    img_model.add(layers.Dense(512, activation='relu'))
    img_model.add(layers.Dense(13, activation='sigmoid'))
    img_model.compile(loss='binary_crossentropy',
          optimizer=optimizers.RMSprop(lr=1e-4),
          metrics=['acc'])
    img_model.fit(X_train, y_train, 
                    batch_size=100, 
                    epochs=10, 
                    validation_data=(X_test, y_test))
    img_model.save('new_product_prediction_model.h5')
    s = 'update finish!'
    return str(s)

In [14]:
#인기 제품 추천 기능
@app.route("/related_kewords/<string:keyword>")
def related_keywords_model(keyword):    
    key = request.files['keyword']
    frequent_itemsets = apriori(key, min_support=0.7, use_colnames=True)
    frequent_itemsets 
    related_keywords = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
    return str(related_keywords)

In [ ]:
if __name__ == "__main__":              
    app.run(host="0.0.0.0", port="20153")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:20153/ (Press CTRL+C to quit)
203.249.127.60 - - [14/Dec/2019 13:16:52] "GET /11st/사과 HTTP/1.1" 200 -


img save finish!


203.249.127.60 - - [14/Dec/2019 13:20:15] "POST /image_recognition HTTP/1.1" 200 -


prediction finish!


...

prediction finish!
img save finish!


203.249.127.60 - - [14/Dec/2019 20:57:31] "POST /image_recognition HTTP/1.1" 200 -


prediction finish!
img save finish!


203.249.127.60 - - [14/Dec/2019 21:04:47] "POST /image_recognition HTTP/1.1" 200 -


prediction finish!
